In [1]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
import json
import re
from pprint import pprint
import csv
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from textatistic import Textatistic
from textblob import TextBlob
from scipy import stats
from scipy.special import inv_boxcox

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True) 
%matplotlib inline
sns.set_style(style='darkgrid')
plt.rcParams["patch.force_edgecolor"]=True

# Import and clean full kiva dataset

In [ ]:
#read in the large 1.4 million entry csv file.
#kiva_big = pd.read_csv('/Users/travis/Downloads/kiva_ds_csv/loans.csv')


In [ ]:
#find places without a translated description and copy over the original so I can drop the description column
kiva_big.DESCRIPTION_TRANSLATED.fillna(kiva_big.DESCRIPTION, inplace=True)

In [ ]:
#Drop rows where there is no description
kiva_big.dropna(axis=0, how='any', subset=['DESCRIPTION'], inplace=True)

In [ ]:
kiva_big.drop(['LOAN_NAME', 'DESCRIPTION', 'NUM_JOURNAL_ENTRIES',
               'NUM_BULK_ENTRIES', 'LOAN_USE','BORROWER_NAMES',
               'BORROWER_PICTURED','VIDEO_ID'],
             axis=1,
             inplace=True)

kiva_big.drop(['CURRENCY'], axis=1, inplace=True)

kiva_big.drop(['CURRENCY_EXCHANGE_COVERAGE_RATE'], axis=1, inplace=True)

kiva_big.drop(['COUNTRY_CODE'], axis=1, inplace=True)

kiva_big.drop(['NUM_LENDERS_TOTAL'], axis=1, inplace=True)

kiva_big.drop(['ACTIVITY_NAME'], axis=1, inplace=True)

In [ ]:
#this gives me percent of rows that have 'TAGS' as null. 55%, keeping this column, can turn into categorical (either yes/no) and use for testing
kiva_big['TAGS'].isnull().mean()

In [ ]:
kiva_big['PARTNER_ID']

In [ ]:
#Export this dataset to a pkl file
#kiva_big.to_pickle('kiva_full.pkl')

In [4]:
kiva_big = pd.read_pickle('kiva_full.pkl')

# Map the entire dataset of loans that were funded

In [51]:
kiva_funded = kiva_big[kiva_big['STATUS'] == 'funded']

In [52]:
#create a plotly choropleth map of countries and the number of loans individuals there received 
counts = kiva_funded.COUNTRY_NAME.value_counts().reset_index()

counts.columns=['COUNTRY_NAME', 'COUNT']

counts['log_count'] = np.log(counts['COUNT'])

data = dict(type = 'choropleth',
             colorscale = 'Viridis',
            reversescale = True,
             locations = counts['COUNTRY_NAME'],
             locationmode = "country names",
             z = counts['log_count'],
             text = counts['COUNT'],
             colorbar = {'title':'Scaled Frequency'})

In [53]:
layout = dict(title = 'Kiva Loan Frequency Data',
             geo = dict(showframe = False,
                       projection = {'type':'equirectangular'}))
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap,validate=False)

In [54]:
kiva_funded['COUNTRY_NAME'].value_counts()

Philippines                         279599
Kenya                               126291
Peru                                 82781
Cambodia                             78046
El Salvador                          53751
Pakistan                             42001
Uganda                               41533
Tajikistan                           40065
Nicaragua                            39359
Ecuador                              31259
Colombia                             26382
Paraguay                             24114
Bolivia                              22932
Vietnam                              20784
Ghana                                20351
Nigeria                              18845
Lebanon                              18619
Mexico                               18305
Samoa                                15512
Rwanda                               15391
Togo                                 14908
India                                14704
Tanzania                             14256
Honduras   

# Create a sample dataset of 50k entries & standardize

In [ ]:
kiva = kiva_big.sample(50000)

In [ ]:
kiva = kiva.reset_index()

In [ ]:
kiva.drop('index', axis=1, inplace=True)

In [ ]:
kiva['DISBURSE_TIME'] = pd.to_datetime(kiva['DISBURSE_TIME'])
kiva['PLANNED_EXPIRATION_TIME'] = pd.to_datetime(kiva['PLANNED_EXPIRATION_TIME'])
kiva['RAISED_TIME'] = pd.to_datetime(kiva['RAISED_TIME'])
kiva['POSTED_TIME'] = pd.to_datetime(kiva['POSTED_TIME'])

In [ ]:
kiva['TIME_DELTA'] = kiva['RAISED_TIME'] - kiva['POSTED_TIME']

In [ ]:
#run to create a days column of number of days it takes a project to get funding.
kiva['DAYS_TO_FUND'] = kiva['TIME_DELTA']/np.timedelta64(1,'D')

In [ ]:
#ensure that the timedelta for Days is a value greater than 0
def cleanDays(row):
    a = row.DAYS_TO_FUND
    if a<0:
        return np.nan
    else:
        return a

In [ ]:
#apply the function cleanDays to the dataframe reassigning negative values to nan. Time must pass forward!
#run when first creating the days column
kiva['DAYS_TO_FUND'] = kiva.apply(cleanDays, axis=1)

In [ ]:
#I found errors in the date posted column, that come with the initial csv.
#I cannot identify those without searching 1x1. but kiva has an upper limit on funding periods
#So anything beyond 60 days I set to nan to catch those errors (less than .5% of this sample is effected)
#This is not guaranteed to catch all errors of this type, but it catches many
#if length of time to get funding is more than 60 days, convert to nan
kiva['DAYS_TO_FUND'][kiva['DAYS_TO_FUND'] >=60] = np.nan

In [ ]:
#create columnns of number of female borrowers and male borrowers per loan and total number of borrowers
kiva['FEMALE_BORROWERS'] = kiva['BORROWER_GENDERS'].str.title().str.count('Female')
kiva['MALE_BORROWERS'] = kiva['BORROWER_GENDERS'].str.title().str.count('Male')
kiva['BORROWERS_TOTAL'] = kiva['FEMALE_BORROWERS'] + kiva['MALE_BORROWERS']
kiva['PRIMARY_GENDER'] = kiva['FEMALE_BORROWERS'] - kiva['MALE_BORROWERS']

In [ ]:
#Extracts the year from the year the loan request was posted
kiva['YEAR'] = kiva['POSTED_TIME'].dt.year

In [ ]:
#Create a column of the month posted, to see if things fund faster in different times of year
kiva['MONTH'] = kiva['POSTED_TIME'].dt.month

In [ ]:
#Set Primary gender to 1 if there are more female borrowers than male borrowers
kiva['PRIMARY_GENDER'][(kiva['FEMALE_BORROWERS'] - kiva['MALE_BORROWERS'] >=0)] = 'Female'
kiva['PRIMARY_GENDER'][((kiva['FEMALE_BORROWERS'] - kiva['MALE_BORROWERS']) <0)] = 'Male'

In [ ]:
kiva['PERCENT_FEMALE']= (kiva['FEMALE_BORROWERS'] / kiva['BORROWERS_TOTAL'])

In [ ]:
kiva['PERCENT_FEMALE_GROUPED'] = 0

#break into clusters
kiva['PERCENT_FEMALE_GROUPED'][(kiva['PERCENT_FEMALE'] == 0)] = 0
kiva['PERCENT_FEMALE_GROUPED'][(kiva['PERCENT_FEMALE'] > 0) & (kiva['PERCENT_FEMALE'] <=.33)] = .25
kiva['PERCENT_FEMALE_GROUPED'][(kiva['PERCENT_FEMALE'] > .33) & (kiva['PERCENT_FEMALE'] <=.66)] = .5
kiva['PERCENT_FEMALE_GROUPED'][(kiva['PERCENT_FEMALE'] > .66) & (kiva['PERCENT_FEMALE'] <=.99)] = .75
kiva['PERCENT_FEMALE_GROUPED'][(kiva['PERCENT_FEMALE'] > .99) & (kiva['PERCENT_FEMALE'] <=1)] = 1


In [ ]:
#convert partnerid column to strings since I want them as categorical data, not numeric
kiva['PARTNER_ID'] = kiva['PARTNER_ID'].apply(str)

In [ ]:
#replace null values in description column with empty strings
kiva['DESCRIPTION_TRANSLATED'] = kiva['DESCRIPTION_TRANSLATED'].fillna('')

In [ ]:
kiva['HAS_TAGS'] = 0
kiva['HAS_TAGS'][kiva['TAGS'].isnull() == True] = 0
kiva['HAS_TAGS'][kiva['TAGS'].isnull() == False] = 1

In [ ]:
#create new column with length of description (word count)
kiva['LEN_DESC'] = kiva['DESCRIPTION_TRANSLATED'].apply(lambda x: len(re.findall(r'\w+', x)))

In [ ]:
def flesch_read(x):
    try:
        s=Textatistic(x)
        return s.flesch_score
    except: return np.nan

In [ ]:
#calculate Flesch Score for descriptions
kiva['FLESCH_SCORE'] = kiva['DESCRIPTION_TRANSLATED'].apply(lambda x: flesch_read(x))

In [ ]:
kiva['LOAN_ID'] = kiva['LOAN_ID'].apply(str)

In [ ]:
#Apply boxcox transformation to Days to Fund and Loan Amount
_,lmb_days = stats.boxcox(kiva['DAYS_TO_FUND'].dropna())
kiva['DAYS_BOX'] = stats.boxcox(kiva['DAYS_TO_FUND'],lmb_days)
_,lmb_loan = stats.boxcox(kiva['LOAN_AMOUNT'].dropna())
kiva['LOAN_BOX'] = stats.boxcox(kiva['LOAN_AMOUNT'],lmb_loan)
#store these lamba's for later re-transformation of the data.
#lmb_days, lmb_loan

In [ ]:
#Create a pickle file of this dataset
#kiva.to_pickle('kiva_423.pkl')

In [ ]:
#Read in a pickle file of a dataset
#kiva = pd.read_pickle('kiva_423.pkl')